# 트랜스포머 (Transformer)


* attention mechanism은 seq2seq의 입력 시퀀스 정보 손실을 보정해주기 위해 사용됨
* attention mechanism을 보정 목적이 아닌, 인코더와 디코더로 구성한 모델이 바로 트랜스포머
* 트랜스포머는 RNN을 사용하지 않고 인코더와 디코더를 설계하였으며, 성능도 RNN보다 우수함

## 포지셔널 인코딩

* 기존의 RNN은 단어의 위치를 따라 순차적으로 입력받아 단어의 위치정보를 활용할 수 있었음
* 트랜스포머의 경우, RNN을 활용하지 않았기 때문에 단어의 위치정보를 다른 방식으로 줄 필요가 있음
* 이를 위해 **각 단어의 임베딩 벡터에 위치 정보들을 더하게 되는데** 이를 포지셔널 인코딩이라 함
* 보통 포지셔널 인코딩은 sin, cos을 이용하여 계산

In [2]:
import numpy as np
import tensorflow as tf
def positional_encoding(dim, sentence_length):
    encoded_vec = np.array([pos / np.power(10000, 2*i / dim) for pos in range(sentence_length) for i in range(dim)])
    encoded_vec[::2] = np.sin(encoded_vec[::2])
    encoded_vec[1::2] = np.cos(encoded_vec[1::2])
    return tf.constant(encoded_vec.reshape([sentence_length, dim]), dtype = tf.float32)

## 레이어 정규화


*  레이어 정규화에서는 텐서의 마지막 차원에 대해 평균과 분산을 구하고, 이 값을 통해 값을 정규화함
*  해당 정규화를 각 층의 연결에 편리하게 적용하기 위해 함수화한 `sublayer_connection()`을 선언

In [3]:
def layer_norm (inputs, eps = 1e-6):
    feature_shape = inputs.get_shape()[-1:]
    mean = tf.keras.backed.mean(inputs, [-1], keepdims = True)
    std = tf.keras.backed.std(inputs, [-1], keepdims = True)
    beta = tf.Variable(tf.zeros(feature_shape), trainable=False)
    gamma = tf.Variable(tf.ones(feature_shape), trainable=False)
    return gamma * (inputs - mean)/ (std+eps) + beta